#Analysis of IMDB database using SQL

##Import required libraries

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [18]:
conn=sqlite3.connect('/content/movies.sqlite') # Create a connection with database
cur=conn.cursor()# Create a cursor object


## Find about the Tables in DataBase

There are two tables in the Database-movies and directors. The columns of the tables are given by the open-source website.



In [22]:
cur.execute("""
SELECT  * FROM movies
  """)
movies=cur.fetchall()

In [26]:
df_movies=pd.DataFrame(movies,columns=[
    'id','original_title','budget','popularity','release_date','revenue','title',
    'vote_avarage','vote_count','overview','tagline','uid','director_id'
])

In [27]:
df_movies.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_avarage,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


In [25]:
cur.execute("SELECT * FROM directors")
directors=cur.fetchall()

In [28]:
df_dir=pd.DataFrame(directors,columns=[
    'name','id','gender','uid','department'
])

In [29]:
df_dir.head()

,name,id,gender,uid,department
0,James Cameron,4762,2,2710,Directing
1,Gore Verbinski,4763,2,1704,Directing
2,Sam Mendes,4764,2,39,Directing
3,Christopher Nolan,4765,2,525,Directing
4,Andrew Stanton,4766,2,7,Directing


In [30]:
df_dir['department'].value_counts()

Directing    2349
Name: department, dtype: int64

Clearly the only value in Department column is Directing. We don't need such column so we would like to delete them.

In [31]:
df_dir.drop(columns=['department'],inplace=True)
df_dir.head()

,name,id,gender,uid
0,James Cameron,4762,2,2710
1,Gore Verbinski,4763,2,1704
2,Sam Mendes,4764,2,39
3,Christopher Nolan,4765,2,525
4,Andrew Stanton,4766,2,7


Count the number of movies in the movies table

In [32]:
len(df_movies["id"].unique())

4773

In [33]:
len(df_movies)

4773

There are no duplicate values in the table.

In [34]:
df_dir['gender'].value_counts()

2    1574
0     625
1     150
Name: gender, dtype: int64

find the data of these three directors : James Cameron, Luc Besson, John Woo

In [44]:
cur.execute("""
    SELECT * FROM directors
      WHERE name in ('James Cameron', 'Luc Besson', 'John Woo')
            """)
cur.fetchall()

[('James Cameron', 4762, 2, 2710, 'Directing'),
 ('John Woo', 4893, 2, 11401, 'Directing'),
 ('Luc Besson', 4949, 2, 59, 'Directing')]

find all the directors with name starting with 'Steven'

In [46]:
cur.execute("""
    SELECT * FROM directors
      WHERE name like 'Steven%'
            """)
cur.fetchall()

[('Steven Spielberg', 4799, 2, 488, 'Directing'),
 ('Steven Soderbergh', 4909, 2, 1884, 'Directing'),
 ('Steven Brill', 5013, 2, 32593, 'Directing'),
 ('Steven Zaillian', 5117, 2, 2260, 'Directing'),
 ('Steven Quale', 5216, 2, 93214, 'Directing'),
 ('Steven Seagal', 5221, 2, 23880, 'Directing'),
 ('Steven E. de Souza', 5390, 2, 1726, 'Directing'),
 ('Steven Shainberg', 5803, 2, 67795, 'Directing'),
 ('Steven R. Monroe', 6713, 2, 88039, 'Directing')]

In [49]:
df_dir[df_dir['name'].str.contains('Steven')]

,name,id,gender,uid
37,Steven Spielberg,4799,2,488
133,Mark Steven Johnson,4895,2,16837
147,Steven Soderbergh,4909,2,1884
251,Steven Brill,5013,2,32593
355,Steven Zaillian,5117,2,2260
358,Robert Stevenhagen,5120,0,64152
454,Steven Quale,5216,2,93214
459,Steven Seagal,5221,2,23880
628,Steven E. de Souza,5390,2,1726
936,George Stevens,5698,2,18738


1 is for females - how many female directors

In [52]:
cur.execute("""
    SELECT count(*) AS 'Female count' FROM directors
      WHERE gender=1
            """)
cur.fetchall()

[(150,)]

In [54]:
len(df_dir[df_dir['gender']==1])

150

the name of the 10th women director

In [56]:
df_dir[df_dir['gender']==1].iloc[9]

name      Angelina Jolie
id                  5086
gender                 1
uid                11701
Name: 324, dtype: object

In [57]:
cur.execute("""
    SELECT name FROM directors
      WHERE gender=1
      LIMIT 10
            """)

cur.fetchall()[9]

('Angelina Jolie',)

what are the 3 most popular movies

In [58]:
cur.execute("""
    SELECT original_title FROM movies
      ORDER BY popularity DESC
      LIMIT 3
            """)

cur.fetchall()

[('Minions',), ('Interstellar',), ('Deadpool',)]

In [61]:
df_movies.sort_values(by='popularity',ascending=False).iloc[:3]['original_title']

546         Minions
95     Interstellar
788        Deadpool
Name: original_title, dtype: object

what is the most awarded average rated movie since jan 1st 2000

In [67]:
cur.execute("""
    SELECT original_title FROM movies
      WHERE release_date>'2000-01-01'
      ORDER BY vote_average DESC
      LIMIT 1
            """)

cur.fetchall()

[('Sardaarji',)]

In [70]:
df_movies[df_movies['release_date']>'2000-01-01'].sort_values(by='vote_avarage').iloc[-1]['original_title']

'Sardaarji'

which movies were directed by Brenda Chapman?

In [71]:
cur.execute("""
    SELECT original_title FROM movies
      WHERE director_id=(SELECT id FROM directors
            WHERE name='Brenda Chapman')
            """)

cur.fetchall()

[('Brave',)]

name of the director who has made most number of movies

In [73]:
cur.execute("""
    SELECT name FROM directors
        JOIN movies ON directors.id=movies.director_id
        GROUP BY director_id
        ORDER BY count(movies.id) DESC
        LIMIT 1
            """)

cur.fetchall()

[('Steven Spielberg',)]

Name of the most bankable director.
Here bankable means who has highest total budget.

In [79]:
cur.execute("""
    SELECT name FROM directors
        JOIN movies ON directors.id=movies.director_id
        GROUP BY director_id
        ORDER BY sum(budget) DESC
        LIMIT 1
            """)

cur.fetchall()

[('Steven Spielberg',)]

tell evrything about top 10 highest budget making movies

In [80]:
cur.execute("""
    SELECT * FROM movies
        JOIN directors ON directors.id=movies.director_id
        ORDER BY (budget) DESC
        LIMIT 10
            """)

cur.fetchall()

[(43614,
  'Pirates of the Caribbean: On Stranger Tides',
  380000000,
  135,
  '2011-05-14',
  1045713802,
  'Pirates of the Caribbean: On Stranger Tides',
  6.4,
  4948,
  "Captain Jack Sparrow crosses paths with a woman from his past, and he's not sure if it's love -- or if she's a ruthless con artist who's using him to find the fabled Fountain of Youth. When she forces him aboard the Queen Anne's Revenge, the ship of the formidable pirate Blackbeard, Jack finds himself on an unexpected adventure in which he doesn't know who to fear more: Blackbeard or the woman from his past.",
  'Live Forever Or Die Trying.',
  1865,
  4775,
  'Rob Marshall',
  4775,
  2,
  17633,
  'Directing'),
 (43598,
  "Pirates of the Caribbean: At World's End",
  300000000,
  139,
  '2007-05-19',
  961000000,
  "Pirates of the Caribbean: At World's End",
  6.9,
  4500,
  'Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Sw

Above data looks pretty weird. We are not able to read anything from this. Let's make a dataframe to get better tablular data.

In [81]:
cur.execute("""
    SELECT * FROM movies
        JOIN directors ON directors.id=movies.director_id
        ORDER BY (budget) DESC
        LIMIT 10
            """)

pd.DataFrame(cur.fetchall())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,43614,Pirates of the Caribbean: On Stranger Tides,380000000,135,2011-05-14,1045713802,Pirates of the Caribbean: On Stranger Tides,6.4,4948,Captain Jack Sparrow crosses paths with a woma...,Live Forever Or Die Trying.,1865,4775,Rob Marshall,4775,2,17633,Directing
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763,Gore Verbinski,4763,2,1704,Directing
2,43604,Avengers: Age of Ultron,280000000,134,2015-04-22,1405403694,Avengers: Age of Ultron,7.3,6767,When Tony Stark tries to jumpstart a dormant p...,A New Age Has Come.,99861,4769,Joss Whedon,4769,2,12891,Directing
3,43607,Superman Returns,270000000,57,2006-06-28,391081192,Superman Returns,5.4,1400,Superman returns to discover his 5-year absenc...,None,1452,4772,Bryan Singer,4772,2,9032,Directing
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766,Andrew Stanton,4766,2,7,Directing
5,43603,Tangled,260000000,48,2010-11-24,591794936,Tangled,7.4,3330,When the kingdom's most wanted-and most charmi...,They're taking adventure to new lengths.,38757,4768,Byron Howard,4768,2,76595,Directing
6,43602,Spider-Man 3,258000000,115,2007-05-01,890871626,Spider-Man 3,5.9,3576,The seemingly invincible Spider-Man goes up ag...,The battle within.,559,4767,Sam Raimi,4767,2,7623,Directing
7,43610,The Lone Ranger,255000000,49,2013-07-03,89289910,The Lone Ranger,5.9,2311,The Texas Rangers chase down a gang of outlaws...,Never Take Off the Mask,57201,4763,Gore Verbinski,4763,2,1704,Directing
8,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765,Christopher Nolan,4765,2,525,Directing
9,43605,Harry Potter and the Half-Blood Prince,250000000,98,2009-07-07,933959197,Harry Potter and the Half-Blood Prince,7.4,5293,"As Harry begins his sixth year at Hogwarts, he...",Dark Secrets Revealed,767,4770,David Yates,4770,2,11343,Directing


## Revenue analysis
Find top 10 revenue making movies

In [84]:
cur.execute("""
    SELECT * FROM movies
        ORDER BY revenue DESC
        LIMIT 10
            """)

pd.DataFrame(cur.fetchall(),columns=[
    'id','original_title','budget','popularity','release_date','revenue','title',
    'vote_avarage','vote_count','overview','tagline','uid','director_id'
])

,id,original_title,budget,popularity,release_date,revenue,title,vote_avarage,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43622,Titanic,200000000,100,1997-11-18,1845034188,Titanic,7.5,7562,"84 years later, a 101-year-old woman named Ros...",Nothing on Earth could come between them.,597,4762
2,43613,The Avengers,220000000,144,2012-04-25,1519557910,The Avengers,7.4,11776,When an unexpected enemy emerges and threatens...,Some assembly required.,24428,4769
3,43625,Jurassic World,150000000,418,2015-06-09,1513528810,Jurassic World,6.5,8662,Twenty-two years after the events of Jurassic ...,The park is open.,135397,4783
4,43641,Furious 7,190000000,102,2015-04-01,1506249360,Furious 7,7.3,4176,Deckard Shaw seeks revenge against Dominic Tor...,Vengeance Hits Home,168259,4794
5,43604,Avengers: Age of Ultron,280000000,134,2015-04-22,1405403694,Avengers: Age of Ultron,7.3,6767,When Tony Stark tries to jumpstart a dormant p...,A New Age Has Come.,99861,4769
6,43721,Frozen,150000000,165,2013-11-27,1274219009,Frozen,7.3,5295,Young princess Anna of Arendelle dreams about ...,Only the act of true love will thaw a frozen h...,109445,4844
7,43628,Iron Man 3,200000000,77,2013-04-18,1215439994,Iron Man 3,6.8,8806,When Tony Stark's world is torn apart by a for...,Unleash the power behind the armor.,68721,4784
8,44143,Minions,74000000,875,2015-06-17,1156730962,Minions,6.4,4571,"Minions Stuart, Kevin and Bob are recruited by...","Before Gru, they had a history of bad bosses",211672,5045
9,43623,Captain America: Civil War,250000000,198,2016-04-27,1153304495,Captain America: Civil War,7.1,7241,"Following the events of Age of Ultron, the col...",Divided We Fall,271110,4781


Name all the directors with the number of movies and revenue

In [87]:
cur.execute("""
    SELECT name,count(title) AS movies_count,sum(revenue) AS total_revenue FROM directors
        JOIN movies ON directors.id=movies.director_id
        GROUP BY director_id
        ORDER BY total_revenue DESC
            """)

pd.DataFrame(cur.fetchall(),columns=[
    'name','movies_count','total_revenue'
])

,name,movies_count,total_revenue
0,Steven Spielberg,27,9147393164
1,Peter Jackson,9,6498642820
2,James Cameron,7,5883569439
3,Michael Bay,12,5832524638
4,Christopher Nolan,8,4227483234
...,...,...,...
2344,David Kellogg,1,0
2345,Andrés Couturier,1,0
2346,Lucile Hadzihalilovic,1,0
2347,Henry Jaglom,1,0
